# Do experiments with cats/dogs dataset

In [1]:
from __future__ import print_function
import argparse
import cv2
import glob
import os
import re
import random
import sys
import time

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tqdm import tqdm

parser = argparse.ArgumentParser(description="Cats/Dogs playground")
parser.add_argument(
    "--config-file",
    default=None,
    metavar="FILE",
    help="path to config file",
    type=str,
    )
parser.add_argument(
        "opts",
        help="Modify config options using the command-line",
        default=None,
        nargs=argparse.REMAINDER,
    )
args = parser.parse_args([])

/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from config import get_cfg_defaults
cfg = get_cfg_defaults()
if args.config_file is not None:
    cfg.merge_from_file(args.config_file)
if args.opts is not None:
    cfg.merge_from_list(args.opts)
cfg.freeze()
sys.path.append(cfg.SYSTEM.BACKBONE_PATH)
print(cfg)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = ','.join(str(i) for i in cfg.SYSTEM.DEVICES)
os.environ['CUDA_VISIBLE_DEVICES'] = device

DATASET:
  TEST: /data/seanyu/cat_dog/dataset/test1/
  TRAIN: /data/seanyu/cat_dog/dataset/train/
EXPERIMENT:
  ABC: 1
  DEF:
    RGB: test
MODEL:
  BACKBONE: R-50-v2
  NORM_USE: bn
  OPTIMIZER: Adam
  USE_PRETRAIN: True
SYSTEM:
  BACKBONE_PATH: /home/seanyu/research/deep-learning-experiments/cats_and_dogs_playground/experimental_stuff
  DEVICES: [0]
  NAME_FLAG: default
  NUM_WORKERS: 2
  QUEUE_SIZE: 50
  RESULT_DIR: ./results/
TRAIN:
  BATCH_SIZE: 32
  EPOCHS: 50
  IMAGE_SIZE: (256, 256, 3)
  LR: 0.0001
  NUM_CLASSES: 2
  NUM_UPDATES: 500
  TRAIN_RATIO: 0.9
  USE_RESNET_PREPROC: True


In [3]:
"""
Customized functions
"""
# model.py
import tensorflow as tf
from backbone import *
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.nn as F

graph_mapping = {
    "R-50-v1":ResNet50,
    "R-101-v1":ResNet101,
    "R-152-v1":ResNet152,
    "R-50-v2":ResNet50V2,
    "R-101-v2":ResNet101V2,
    "R-152-v2":ResNet152V2,
    "R-50-xt":ResNeXt50,
    "R-101-xt":ResNeXt101}

class Main_model(models.Model):
    def __init__(self, num_class=2, **kwargs):
        super(Main_model, self).__init__(**kwargs)
        
        self.gap_layer = layers.GlobalAveragePooling2D(name="GAP_layer")
        self.output_layer = layers.Dense(units=num_class)
        
        
    def call(self, x, training=True):
        x = self.gap_layer(x)
        output = self.output_layer(x)       
        return F.softmax(output)

def entropy_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred)

In [4]:
tf.keras.backend.clear_session()

module = graph_mapping[cfg.MODEL.BACKBONE](input_shape=(256,256,3), include_top=False, weights="imagenet", norm_use="bn")
"""
xx = layers.GlobalAveragePooling2D()(module.output)
xx = layers.Dense(units=2, activation="softmax")(xx)
model = models.Model(inputs=[module.input], outputs=[xx])
"""
m_head = Main_model(num_class=2)(module.output)
model = models.Model(inputs=[module.input], outputs=[m_head])



In [5]:
# sub-functions
from model import build_model, make_optimizer, preproc
from augment import Augmentation_Setup
from data_generator import GetDataset, Customized_dataloader

In [6]:
"""  Get data """
image_train_list = glob.glob(cfg.DATASET.TRAIN + '*.jpg')
image_test_list = glob.glob(cfg.DATASET.TEST + '*.jpg')

df_train = pd.DataFrame({'img_path': image_train_list})
df_test = pd.DataFrame({'img_path': image_test_list})

df_train['cate'] = df_train.img_path.apply(os.path.basename)
df_train['cate'] = [i.split(".")[0] for i in list(df_train.cate)]
df_train.cate = df_train.cate.replace({'dog': 0, 'cat': 1})

df_train_0, df_val_0 = train_test_split(df_train[df_train['cate'] == 0], test_size = 1-cfg.TRAIN.TRAIN_RATIO)
df_train_1, df_val_1 = train_test_split(df_train[df_train['cate'] == 1], test_size = 1-cfg.TRAIN.TRAIN_RATIO)
df_val = pd.concat((df_val_0, df_val_1)).reset_index(drop = True)

del df_val_0, df_val_1

USE_RESNET_PREPROC = cfg.TRAIN.USE_RESNET_PREPROC
dtrain = GetDataset(df_list=df_train,
                    class_id=0, n_classes=2,
                    f_input_preproc=preproc if not USE_RESNET_PREPROC else tf.keras.applications.resnet50.preprocess_input,
                    augmentation=Augmentation_Setup.augmentation, 
                    onehot= True, 
                    image_size=cfg.TRAIN.IMAGE_SIZE)

dvalid = GetDataset(df_list=df_val, 
                    class_id=0, n_classes=2,
                    f_input_preproc=preproc if not USE_RESNET_PREPROC else tf.keras.applications.resnet50.preprocess_input,
                    augmentation=None, 
                    onehot= True, 
                    image_size=cfg.TRAIN.IMAGE_SIZE)

valid_gen = Customized_dataloader([dvalid], batch_size_per_dataset=16, num_workers=1)
x_val, y_val = [], []
for _ in tqdm(range(100)):
    a,b = next(valid_gen)
    x_val.append(a)
    y_val.append(b)
x_val = np.concatenate(x_val)
y_val = np.concatenate(y_val)
valid_gen.stop_worker()

print(x_val.shape)
print(y_val.shape)
print(y_val.sum(axis=0))

  1%|          | 1/100 [00:00<00:17,  5.64it/s]

workers ready


100%|██████████| 100/100 [00:12<00:00,  8.29it/s]


all_worker_stop
(1600, 256, 256, 3)
(1600, 2)
[815. 785.]


In [7]:
train_gen = Customized_dataloader([dtrain], 
                                  batch_size_per_dataset=cfg.TRAIN.BATCH_SIZE // cfg.TRAIN.NUM_CLASSES, 
                                  num_workers=cfg.SYSTEM.NUM_WORKERS, 
                                  queue_size=cfg.SYSTEM.QUEUE_SIZE)

workers ready


In [8]:
"""
model = build_model(backbone=cfg.MODEL.BACKBONE, 
                    norm_use=cfg.MODEL.NORM_USE, 
                    weights="imagenet" if cfg.MODEL.USE_PRETRAIN else None)
"""
optim = make_optimizer(cfg)
model.compile(loss=entropy_loss,#'categorical_crossentropy', 
              metrics=["accuracy"], 
              optimizer=optim)
model.summary()

cb_list = [tf.keras.callbacks.ReduceLROnPlateau(factor=0.5,
                                            patience=4,
                                            min_lr=1e-12),
      ]
model.fit_generator(train_gen,
                    epochs=cfg.TRAIN.EPOCHS,
                    steps_per_epoch=cfg.TRAIN.NUM_UPDATES, 
                    validation_data=(x_val, y_val),
                    callbacks=cb_list
                    )

train_loss = model.history.history['loss']
valid_loss = model.history.history['val_loss']
train_acc = model.history.history['acc']
valid_acc = model.history.history['val_acc']

plt.figure(figsize=(8,6))
plt.plot(range(len(train_loss)), train_loss, label='train_loss')
plt.plot(range(len(valid_loss)), valid_loss, label='valid_loss')
plt.legend()
plt.savefig(os.path.join("results", "exp_" + cfg.SYSTEM.NAME_FLAG + "_loss.png"))

plt.figure(figsize=(8,6))
plt.plot(range(len(train_acc)), train_acc, label='train_accuracy')
plt.plot(range(len(valid_acc)), valid_acc, label='valid_accuracy')
plt.legend()
plt.savefig(os.path.join("results", "exp_" + cfg.SYSTEM.NAME_FLAG + "_acc.png"))

result_df = pd.DataFrame({"train_loss":train_loss,
                          "valid_loss":valid_loss,
                          "train_acc":train_acc,
                          "valid_acc":valid_acc
                         })
result_df.to_csv(os.path.join("results", "exp_" + cfg.SYSTEM.NAME_FLAG + "_result.csv"), index=False)
print("All Done")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 130, 130, 64) 0           conv1_conv[0][0]                 
__________________________________________________________________________________________________
pool1_pool

Epoch 1/50
500/500 [==============================] - 112s 225ms/step - loss: 0.1424 - acc: 0.9470 - val_loss: 0.2330 - val_acc: 0.9244
Epoch 2/50
500/500 [==============================] - 124s 248ms/step - loss: 0.1379 - acc: 0.9499 - val_loss: 0.1390 - val_acc: 0.9450
Epoch 3/50
500/500 [==============================] - 114s 227ms/step - loss: 0.1216 - acc: 0.9544 - val_loss: 0.1356 - val_acc: 0.9575
Epoch 4/50
500/500 [==============================] - 103s 205ms/step - loss: 0.0916 - acc: 0.9675 - val_loss: 0.0945 - val_acc: 0.9600
Epoch 5/50
500/500 [==============================] - 124s 248ms/step - loss: 0.0894 - acc: 0.9677 - val_loss: 0.0975 - val_acc: 0.9644
Epoch 6/50
500/500 [==============================] - 115s 230ms/step - loss: 0.1166 - acc: 0.9629 - val_loss: 0.1568 - val_acc: 0.9400
Epoch 7/50
500/500 [==============================] - 103s 206ms/step - loss: 0.0928 - acc: 0.9701 - val_loss: 0.0984 - val_acc: 0.9656
Epoch 8/50
500/500 [============================

NameError: name 'plt' is not defined

In [ ]:
model.summary()